# **Using Encoder-only LLM to build a Multi-Task Model (Classification + Labeling)**

**Author: Partha Seetala**

Video Tutorial: [https://youtu.be/UJZ4HGLnSMU](https://youtu.be/UJZ4HGLnSMU)

## **Install HuggingFace Transformers Library**

In [ ]:
!pip install --upgrade transformers datasets torchinfo scikit-learn
!pip install "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
  

## **Import required modules**

In [ ]:
import os
os.environ["WANDB_MODE"] = "offline"   # disable W&B prompts

import torch
import torch.nn.functional as F
import numpy as np
from datasets import Dataset
from datasets import load_dataset
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import random
import json
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from huggingface_hub import login

## **Load Google Drive into Colab to load training dataset and cached models**

In [ ]:
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive')

USECASE_NAME = "s3e2-classification-and-labeling"

logmsg_data = "s3e2-linux-log-messages.csv"
k8smsg_data = "s3e2-k8s-log-messages.csv"

DATASET_TYPE = "kubernetes" # "kubernetes" | "logmsg"

if DATASET_TYPE == "logmsg":
    selected_dataset = logmsg_data
elif DATASET_TYPE == "kubernetes":
    selected_dataset = k8smsg_data
else:
    raise ValueError(f"Unknown dataset type: {DATASET_TYPE}")

DATASET_NAME = Path(selected_dataset).stem

MODEL_NAME = "bert-base-uncased"
max_seq_len = 128

ROOTDIR = '/content/drive/MyDrive/cidl'

HF_TOKEN_FILEPATH = os.path.join(ROOTDIR, "hf.token")
DATASET_DIR = os.path.join(ROOTDIR, 'datasets')  # ~/cidl/datasets
PRETRAINED_MODEL_DIR = os.path.join(ROOTDIR, 'models', 'pretrained', 'bert')  # ~/cidl/models/pretrained/bert
FINETUNED_MODEL_DIR = os.path.join(ROOTDIR, 'models', 'finetuned', 'bert', USECASE_NAME, DATASET_NAME)  # ~/cidl/models/finetuned/bert/<usecase-name>/<dataset-name>

def load_hugging_face_token(hf_token_filepath=HF_TOKEN_FILEPATH):
    if os.path.exists(hf_token_filepath):
        with open(hf_token_filepath, "r") as f:
            hf_token = f.read().strip()

        if hf_token.startswith("hf_"):
            # Login and set environment variable
            os.environ["HF_TOKEN"] = hf_token
            #login(token=hf_token, add_to_git_credential=False)
        else:
            print(f"Malformed Hugging Face token file at: {hf_token_filepath}")
    else:
        print(f"Hugging Face token file not found at: {hf_token_filepath}")

for dirpath in [DATASET_DIR, PRETRAINED_MODEL_DIR, FINETUNED_MODEL_DIR]:
    os.makedirs(dirpath, exist_ok=True)

load_hugging_face_token(HF_TOKEN_FILEPATH)

print("Dataset directory .................................... ", DATASET_DIR)
print("Location where pretrained model will be downloaded ... ", PRETRAINED_MODEL_DIR)
print("Location where finetuned model will be stored ........ ", FINETUNED_MODEL_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset directory ....................................  /content/drive/MyDrive/cidl/datasets
Location where pretrained model will be downloaded ...  /content/drive/MyDrive/cidl/models/pretrained/bert
Location where finetuned model will be stored ........  /content/drive/MyDrive/cidl/models/finetuned/bert/s3e2-classification-and-labeling/s3e2-k8s-log-messages


# **STEP 1: Build our custom Classificaton + Labeling Model**

**Download/Load Pre-trained Models**

In [ ]:
def download_pretrained_encoder_only_model(model_dir, model_name):
    # Download the pre-trained Encoder-only Transformer model and the pre-trained Tokenizer for that model
    AutoTokenizer.from_pretrained(model_name, cache_dir=model_dir)
    AutoModel.from_pretrained(model_name, cache_dir=model_dir)

print("Downloading pretrained Encoder-only LLM model '{}' into '{}".format(MODEL_NAME, PRETRAINED_MODEL_DIR))
download_pretrained_encoder_only_model(model_dir=PRETRAINED_MODEL_DIR, model_name=MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Implementation of our custom Classificaiton + Label MultiTask Model and other utility functions**

In [ ]:
class ClassificationLabelingModel(torch.nn.Module):
    def __init__(self, model_name, cache_dir, label_names, device=None):
        super().__init__()
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = device

        self.model_name = model_name
        self.num_labels = len(label_names)
        self.label_names = label_names

        self.base_model = AutoModel.from_pretrained(pretrained_model_name_or_path=model_name, cache_dir=cache_dir)
        self.base_model = self.base_model.to(device)

        # Seq Len = S (512)
        # Dimensions = D (768)
        # [SxD]

        hidden_size = self.base_model.config.hidden_size  # D

        # Urgency FFN head --> Urgent/Normal Classification
        self.urgency_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size),   # [DxD]
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, 2) # [Dx2]
        ).to(device)
        self.urgency_loss_fn = torch.nn.CrossEntropyLoss()  # Softmax()

        # Multilabel FFN head -- Multi-label Head
        self.multilabel_classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, self.num_labels)  # [DxL]
        ).to(device)
        self.multilabel_loss_fn = torch.nn.BCEWithLogitsLoss() # -> Sigmoid()

    def forward(self, input_ids=None, attention_mask=None, urgency_label=None, multilabel_targets=None):
        # "Pod is sitting in a crashloop" => [CLS, pod, is, sitting, in, a, crashloop, PAD, PAD]
        # input_ids => [1, 23, 45, 67, 99, 3, 17, 100, 100]
        # atten_mask=> [0, 1, 1, 1, 1, 1, 1, 0, 0]

        output = self.base_model(input_ids=input_ids, attention_mask=attention_mask)  # [SxD]
        cls_output = output.last_hidden_state[:, 0]  # [CLS] token representation

        # CLS -> [1xD]

        # Urgency Classifier FFN2
        #   [1xD]*[DxD] => [1xD] -> ReLU() -> [1xD]*[Dx2] => [1x2]  [{0=urgent},{1=normal}]
        urgency_logits = self.urgency_classifier(cls_output)  # Ypred_urgency_classifier

        # Multi-labeler FFN2
        #  [1xD]*[DxD] => [1xD] -> ReLU() -> [1xD]*[DxL] => [1xL]
        multilabel_logits = self.multilabel_classifier(cls_output)  # Ypred_multilabeler

        if urgency_label is not None and multilabel_targets is not None:
            # We are training here
            loss_urgency    = self.urgency_loss_fn(urgency_logits, urgency_label)
            loss_multilabel = self.multilabel_loss_fn(multilabel_logits, multilabel_targets.float())
            loss = loss_urgency + loss_multilabel
        else:
            # We are running inference here
            loss = None

        return {
            "loss": loss,
            "urgency_logits": urgency_logits,
            "multilabel_logits": multilabel_logits
        }

    def save_pretrained(self, save_directory):
        os.makedirs(save_directory, exist_ok=True)

        # Save updated weights of Base Encoder-only Transformer Model
        self.base_model.save_pretrained(save_directory)

        # Save weights of custom classifier heads
        torch.save({
            "urgency_classifier": self.urgency_classifier.state_dict(),
            "multilabel_classifier": self.multilabel_classifier.state_dict()
        }, os.path.join(save_directory, "custom_classifier_heads.bin"))

        # Hugging Face-style config
        hf_config = AutoConfig.from_pretrained(self.model_name)
        hf_config.model_name = self.model_name
        hf_config.num_labels = self.num_labels
        hf_config.label_names = self.label_names
        hf_config.save_pretrained(save_directory)

    @classmethod
    def from_pretrained(cls, load_directory):
        # Load config
        hf_config = AutoConfig.from_pretrained(load_directory)

        # Initialize model
        model = cls(
            model_name=hf_config._name_or_path,
            cache_dir=None,
            label_names=hf_config.label_names)

        model.base_model = AutoModel.from_pretrained(load_directory)

        # Load custom classifier weights
        head_weights = torch.load(os.path.join(load_directory, "custom_classifier_heads.bin"), map_location="cpu")
        model.urgency_classifier.load_state_dict(head_weights["urgency_classifier"])
        model.multilabel_classifier.load_state_dict(head_weights["multilabel_classifier"])

        model.eval()

        config = {
            'model_name': hf_config._name_or_path,
            'num_labels': hf_config.num_labels,
            'label_names': hf_config.label_names
        }

        return model, config

def display_classification_and_labeling_model_summary(model):
    print("\n{:<60} {:<20} {:>15} {:>12}".format("Layer (type)", "Shape (S×D)", "Param #", "Trainable"))
    print("=" * 115)

    total_params = 0
    trainable_params = 0
    total_bytes = 0

    for name, param in model.named_parameters():
        shape = " × ".join(map(str, param.shape))
        num_params = param.numel()
        size_bytes = param.element_size() * num_params
        total_params += num_params
        total_bytes += size_bytes
        if param.requires_grad:
            trainable_params += num_params
        print(f"{name:<60} {shape:<20} {num_params:>15,} {str(param.requires_grad):>12}")

    print("=" * 115)
    print(f"Total Parameters         : {total_params:,}")
    print(f"Trainable Parameters     : {trainable_params:,}")
    print(f"Non-trainable Parameters : {total_params - trainable_params:,}")
    print(f"Model Size (approx)      : {total_bytes / (1024**2):.2f} MiB  ({total_bytes / (1024**3):.2f} GiB)")


def build_classification_and_labeling_model(model_dir, model_name, label_names, device=None):
    return ClassificationLabelingModel(model_name=model_name, cache_dir=model_dir, label_names=label_names, device=device)

def finetune_classification_and_labeling_model(model, model_dir, train_dataset, val_dataset, epochs=3):
    def collate_fn(batch):
        return {
            "input_ids": torch.tensor([x["input_ids"] for x in batch], dtype=torch.long),
            "attention_mask": torch.tensor([x["attention_mask"] for x in batch], dtype=torch.long),
            "urgency_label": torch.tensor([x["urgency_label"] for x in batch], dtype=torch.long),
            "multilabel_targets": torch.tensor([x["multilabel_targets"] for x in batch], dtype=torch.float)
        }

    output_dir = os.path.join(model_dir, "checkpoints")
    log_dir    = os.path.join(model_dir, "logs")

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=epochs,
        logging_steps=10,
        logging_dir=log_dir,
        save_total_limit=min(epochs, 3),
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn
    )
    trainer.train()

def save_classification_and_labeling_model(tokenizer, model, finetuned_model_dir):
    print("Saving finetuned model to dir: ", finetuned_model_dir)
    model.save_pretrained(finetuned_model_dir)
    tokenizer.save_pretrained(finetuned_model_dir)

def load_classification_and_labeling_model(finetuned_model_dir):
    print("Loading model from:", finetuned_model_dir)
    model, config = ClassificationLabelingModel.from_pretrained(finetuned_model_dir)
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_dir)
    return tokenizer, model, config

def classify_and_label_support_issue(tokenizer, model, sentence, label_names):

    device = next(model.parameters()).device
    model.to(device)
    model.eval()

    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
    urgency = torch.argmax(F.softmax(outputs["urgency_logits"], dim=1)).item()
    multilabel_probs = torch.sigmoid(outputs["multilabel_logits"]).squeeze().tolist()
    if isinstance(multilabel_probs, float):
        multilabel_probs = [multilabel_probs]
    labels = [label for label, prob in zip(label_names, multilabel_probs) if prob >= 0.5]
    return ("Urgent" if urgency == 1 else "Normal"), labels


# **STEP 2: Finetune our custom model by training with task-specific data**

**Prepare Dataset for fine-tuning model**

In [ ]:
def prepare_dataset_for_finetuning(dataset_csv, tokenizer, max_seqlen=128):
    # FAST PATH: check if we have a previously pre-processed dataset instead of parsing CSV again
    preprocessed_dir = os.path.splitext(dataset_csv)[0] + "-preprocessed"
    dataset_cache_file = os.path.join(preprocessed_dir, "dataset.ds")
    labels_file = os.path.join(preprocessed_dir, "labels.bin")

    # FAST PATH: Check if preprocessed files exist
    if all(os.path.exists(f) for f in [dataset_cache_file, labels_file]):
        dataset = load_from_disk(dataset_cache_file)
        with open(labels_file, "r") as f:
            labels = json.load(f)
        train_test = dataset.train_test_split(test_size=0.1)
        print("Loaded pre-processed dataset from prior cache at {}".format(preprocessed_dir))
        return train_test["train"], train_test["test"], labels

    df = pd.read_csv(dataset_csv)

    # Make all column names lowercase for case-insensitive access
    df.columns = df.columns.str.lower()

    # Ensure 'text', 'urgency', and 'labels' columns exist
    required_cols = {"text", "urgency", "labels"}
    missing_cols = required_cols - set(df.columns)
    if missing_cols:
        raise ValueError(f"Missing required columns in CSV: {missing_cols}")

    # Normalize column values
    df["text"] = df["text"].astype(str)
    df["urgency"] = df["urgency"].astype(str).str.lower().str.strip()
    df["urgency_label"] = df["urgency"].astype(str).str.strip().str.lower().map({"normal": 0, "urgent": 1})

    # Drop rows where urgency_label could not be mapped
    df = df[df["urgency_label"].notnull()]
    df["urgency_label"] = df["urgency_label"].astype(int)

    # parse labels column
    df['labels'] = df['labels'].apply(eval)

    # text1 -> labels=["tag1", "tag2"] -> [1, 1, 0]
    # text2 -> labels=["tag3"] -> [0, 0, 1]
    # text3 -> labels=["tag2", "tag3"] -> [0, 1, 1]
    mlb = MultiLabelBinarizer()
    multilabel = mlb.fit_transform(df['labels'])  # text1 -> [1, 1, 0] | text2 -> [0, 0, 1] | text3 -> [0, 1, 1]
    label_names = list(mlb.classes_)  # ["tag1", "tag2", "tag3"]
    num_labels = len(label_names)     # 3

    multilabel_df = pd.DataFrame(multilabel, columns=[f"label_{l}" for l in label_names])
    df = pd.concat([df.reset_index(drop=True), multilabel_df], axis=1)

    tokenized = tokenizer(
        df['text'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_seqlen,
        return_tensors='pt'
    )

    dataset = Dataset.from_dict({
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "urgency_label": df["urgency_label"].tolist(),
        "multilabel_targets": multilabel.tolist()
    })

    # SAVE in preprocessed cache for future use
    os.makedirs(preprocessed_dir, exist_ok=True)
    dataset.save_to_disk(dataset_cache_file)
    with open(labels_file, "w") as f:
        json.dump(label_names, f)

    train_test = dataset.train_test_split(test_size=0.1)
    return train_test["train"], train_test["test"], label_names

DATASET_FILE_PATH = os.path.join(DATASET_DIR, selected_dataset)

print("Loading training dataset from: ", DATASET_FILE_PATH)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=PRETRAINED_MODEL_DIR)

train_dataset, val_dataset, label_names = prepare_dataset_for_finetuning(dataset_csv=DATASET_FILE_PATH, tokenizer=tokenizer, max_seqlen=max_seq_len)

print("# messages for training: {}".format(len(train_dataset)))
print("# messages for validation: {}".format(len(val_dataset)))
print("unique labels: ({}) {}".format(len(label_names), label_names))

Loading training dataset from:  /content/drive/MyDrive/cidl/datasets/s3e2-k8s-log-messages.csv
Loaded pre-processed dataset from prior cache at /content/drive/MyDrive/cidl/datasets/s3e2-k8s-log-messages-preprocessed
# messages for training: 45018
# messages for validation: 5002
unique labels: (30) ['API-Server', 'Auth', 'Business-Impact', 'ConfigMap', 'Configuration', 'Container-Registry', 'DaemonSet', 'Database', 'Deployment', 'Deployment-Issues', 'Development', 'Infrastructure', 'Ingress', 'Networking', 'Node-Health', 'PVC', 'Performance', 'Pod', 'RBAC', 'Resource-Issues', 'Resource-Management', 'Scaling', 'Scheduling', 'Secret', 'Security', 'Service', 'Service-Discovery', 'StatefulSet', 'Storage', 'Traffic-Management']


**Fine-tune Model using this dataset**

In [ ]:
model = build_classification_and_labeling_model(model_dir=PRETRAINED_MODEL_DIR, model_name=MODEL_NAME, label_names=label_names)

finetune_classification_and_labeling_model(model=model, model_dir=FINETUNED_MODEL_DIR, train_dataset=train_dataset, val_dataset=val_dataset, epochs=3)

Epoch,Training Loss,Validation Loss
1,0.000300,0.000555
2,0.000200,0.000109
3,0.000000,0.000052


**Show summary of model architecture**

In [ ]:
display_classification_and_labeling_model_summary(model)


Layer (type)                                                 Shape (S×D)                  Param #    Trainable
base_model.embeddings.word_embeddings.weight                 30522 × 768               23,440,896         True
base_model.embeddings.position_embeddings.weight             512 × 768                    393,216         True
base_model.embeddings.token_type_embeddings.weight           2 × 768                        1,536         True
base_model.embeddings.LayerNorm.weight                       768                              768         True
base_model.embeddings.LayerNorm.bias                         768                              768         True
base_model.encoder.layer.0.attention.self.query.weight       768 × 768                    589,824         True
base_model.encoder.layer.0.attention.self.query.bias         768                              768         True
base_model.encoder.layer.0.attention.self.key.weight         768 × 768                    589,824         True


**Save out fine-tuned Model**

In [ ]:
save_classification_and_labeling_model(tokenizer, model, FINETUNED_MODEL_DIR)


Saving finetuned model to dir:  /content/drive/MyDrive/cidl/models/finetuned/bert/s3e2-classification-and-labeling/s3e2-k8s-log-messages


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# **STEP 3: Use fine-tuned custom model to classify and label support issues**

In [ ]:
import time

dataset_type = "kubernetes"

if dataset_type == "logmsg":
    issues = [
        "The VPN connection keeps timing out and users are unable to login.",
        "Routine filesystem scan completed successfully.",
        "Previously corrupted filesystem scan completed successfully.",
        "hey I tried to mount a file system at /dev/foobar, however, it fails me everytime",
        "/dev/barfoo can't access it! what should I do?",
    ]
elif dataset_type == "kubernetes":
    issues = [
        'Back-off restarting failed container',
        'CrashLoopBackOff: Container failed to start repeatedly',
        'Error: ImagePullBackOff',
        'Failed to pull image "nginx:latest": rpc error: code = Unknown desc = failed to resolve image',
        'container "app" terminated with exit code 137',
        'Liveness probe failed: HTTP probe failed with statuscode: 503',
        'Readiness probe failed: dial tcp 10.0.0.42:8080: connect: connection refused',
        'ReplicaSet "webapp-5f8657f9b6" has timed out progressing',
        'Deployment "api" exceeded its progress deadline',
        'FailedCreate: Error creating: pods "web-xyz-" is forbidden: exceeded quota',
        'no endpoints available for service "my-backend"',
        'Failed to resolve DNS name: kube-dns.kube-system.svc.cluster.local',
        'NetworkPolicy denied ingress traffic to pod "web-123"',
        'Failed to attach volume "pvc-1234abcd" to node "node-1": timeout while waiting for mount',
        'MountVolume.MountDevice failed for volume "data": rpc error: code = Internal desc = mount failed',
        'pod has unbound immediate PersistentVolumeClaims',
        'Node "gke-node-123" not ready: NetworkUnavailable',
        '0/5 nodes are available: 2 Insufficient cpu, 1 Insufficient memory',
        'Pod eviction due to node pressure: memory pressure',
        'User "system:serviceaccount:default:my-sa" is forbidden: User cannot list resource "pods" in API group "" in the namespace "default"'
    ]
else:
    raise ValueError(f"Unknown dataset type: {dataset_type}")


tokenizer, model, config = load_classification_and_labeling_model(FINETUNED_MODEL_DIR)
label_names = config['label_names']


Loading model from: /content/drive/MyDrive/cidl/models/finetuned/bert/s3e2-classification-and-labeling/s3e2-k8s-log-messages


In [ ]:

for issue in issues:
    start_time = time.time()

    urgency, labels = classify_and_label_support_issue(tokenizer, model, issue, label_names=label_names)

    elapsed_ms = round((time.time() - start_time) * 1000)

    urgency_str = f"\033[91mUrgent\033[0m" if urgency == "Urgent" else urgency # show urgent messages in RED

    print(f"ISSUE: {issue}\n → Time: {elapsed_ms} ms\n → Urgency: {urgency_str}\n → Labels: {labels}\n")


ISSUE: Back-off restarting failed container
 → Time: 309 ms
 → Urgency: Urgent
 → Labels: ['Node-Health', 'Pod', 'Resource-Issues']

ISSUE: CrashLoopBackOff: Container failed to start repeatedly
 → Time: 45 ms
 → Urgency: Urgent
 → Labels: ['Node-Health', 'Pod', 'Resource-Issues']

ISSUE: Error: ImagePullBackOff
 → Time: 27 ms
 → Urgency: Urgent
 → Labels: ['Configuration', 'Container-Registry', 'Pod']

ISSUE: Failed to pull image "nginx:latest": rpc error: code = Unknown desc = failed to resolve image
 → Time: 54 ms
 → Urgency: Urgent
 → Labels: ['Configuration', 'Container-Registry', 'Pod']

ISSUE: container "app" terminated with exit code 137
 → Time: 28 ms
 → Urgency: Normal
 → Labels: ['Pod', 'Resource-Issues']

ISSUE: Liveness probe failed: HTTP probe failed with statuscode: 503
 → Time: 47 ms
 → Urgency: Urgent
 → Labels: ['Configuration', 'Scheduling']

ISSUE: Readiness probe failed: dial tcp 10.0.0.42:8080: connect: connection refused
 → Time: 49 ms
 → Urgency: Urgent
 → Label